In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
# thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
thispath = os.getcwd()
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.metric.metric import MetricCalculator
from mesostat.utils.qt_helper import gui_fname, gui_fnames, gui_fpath
from mesostat.utils.hdf5_io import DataStorage
from mesostat.utils.pandas_helper import pd_query

from lib.gallerosalas.data_fc_db_raw import DataFCDatabase
import lib.analysis.triplet_analysis.mousewise as mousewise
import lib.analysis.triplet_analysis.calc_reader_mousewise as calc_reader

%load_ext autoreload
%autoreload 2

In [ ]:
params = {}
# params['root_path_data']  = gui_fpath("Path to data collection",  './')
params['root_path_data'] = '/media/alyosha/Data/TE_data/yasirdata_raw/'
# params['root_path_data'] = '/home/alyosha/data/yasirdata_raw/'
# params['root_path_data'] = '/media/aleksejs/DataHDD/work/data/yasir/yasirdata_raw'

In [ ]:
dataDB = DataFCDatabase(params)

In [ ]:
dataDB.calc_shortest_distances()

In [ ]:
print('mice', dataDB.mice)
print('nSessions', len(dataDB.sessions))
print('datatypes', dataDB.get_data_types())
print('nChannel', dataDB.get_nchannels('mou_5'))

In [ ]:
h5outname = 'gallerosalas_result_higher_order_df.h5'

In [ ]:
mc = MetricCalculator(serial=True, verbose=False, nCore=4)

In [ ]:
dataDB.channelAreasDF

## All - Distribution - Nosession

In [ ]:
prefixPath = '/media/alyosha/Data/TE_data/calculations/pr2/yasir-tex/'

pwdH5data = prefixPath + 'pr2_gallerosalas_multimouse_nbin_2_data.h5'
pwdH5rand = prefixPath + 'pr2_gallerosalas_multimouse_nbin_2_rand.h5'

In [ ]:
intervNames = dataDB.get_interval_names()
#trialTypes = ['Hit', 'CR', 'Miss', 'FA']
trialTypes = ['Hit', 'CR']

In [ ]:
dfSummary = calc_reader.summary_df(pwdH5data)

In [ ]:
dfSummaryRand = calc_reader.summary_df(pwdH5rand)

In [ ]:
dfSummary

In [ ]:
mousewise.plot_cdf(pwdH5data, dfSummary)

In [ ]:
rezTest = mousewise.plot_violin_test(pwdH5data, pwdH5rand, dfSummary, dfSummaryRand)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(pd_query(rezTest, {'atom': 'syn'}).drop(['key', 'metric'], axis=1))

In [ ]:
mousewise.barplot_avg(dataDB, pwdH5data, dfSummary, 'intervName', intervNames)

In [ ]:
mousewise.barplot_avg(dataDB, pwdH5data, dfSummary, 'trialType', trialTypes)

In [ ]:
mousewise.plot_top_singlets(dataDB, pwdH5data, dfSummary)

In [ ]:
mousewise.plot_singlets_brainplot(dataDB, pwdH5data, dfSummary, 'intervName', intervNames)

In [ ]:
mousewise.plot_singlets_brainplot(dataDB, pwdH5data, dfSummary, 'trialType',
                                  trialTypes)

In [ ]:
mousewise.plot_singlets_brainplot_mousephase_subpre(dataDB, pwdH5data, dfSummary)

In [ ]:
mousewise.plot_singlets_brainplot_mousephase_submouse(dataDB, pwdH5data, dfSummary)

In [ ]:
mousewise.plot_top_triplets(dataDB, pwdH5data, dfSummary, nTop=20)#, dropChannels=['BC'])

In [ ]:
mousewise.plot_2D_avg(dataDB, pwdH5data, dfSummary, 'intervName', intervNames,
                      dropChannels=[16, 26], avgAxis=2)

In [ ]:
mousewise.plot_2D_avg(dataDB, pwdH5data, dfSummary, 'trialType', trialTypes,
                      dropChannels=[16, 26], avgAxis=2)

In [ ]:
for trgChName in ['SSp-bfd', 'VISrl', 'AUDp']:
    mousewise.plot_2D_target(dataDB, pwdH5data, dfSummary, trgChName,
                         'intervName', intervNames, dropChannels=[16, 26])
    mousewise.plot_2D_target(dataDB, pwdH5data, dfSummary, trgChName,
                         'trialType', trialTypes, dropChannels=[16, 26])

In [ ]:
for trgChName in ['SSp-bfd', 'VISrl', 'AUDp']:
    mousewise.plot_2D_target_mousephase_subpre(dataDB, pwdH5data, dfSummary,
                                              trgChName, dropChannels=[16, 26])

In [ ]:
for trgChName in ['SSp-bfd', 'VISrl', 'AUDp']:
    mousewise.plot_2D_target_mousephase_submouse(dataDB, pwdH5data, dfSummary,
                                                trgChName, dropChannels=[16, 26])

In [ ]:
pid_joint.plot_2D_bytarget_synergy_cluster(dataDB, pwdAllH5_2, dfSummary, 'BC',
                                           dropChannels=None, clusterParam=0.001, dropWeakChannelThr=0.1)

In [ ]:
pid_joint.plot_unique_top_pairs(dataDB, pwdAllH5_2, dfSummary, nTop=20, dropChannels=None)

# Consistency

In [ ]:
pid_joint.plot_consistency_bymouse(pwdAllH5_2, dfSummary, dropChannels=None, kind='fisher', limits=[0, 1])

In [ ]:
pid_joint.plot_consistency_byphase(pwdAllH5_2, dfSummary, dropChannels=None,
                                   kind='fisher', limits=[0, 1], datatype='bn_trial')
pid_joint.plot_consistency_byphase(pwdAllH5_2, dfSummary, dropChannels=None, 
                                   kind='fisher', limits=[0, 1], datatype='bn_session')

In [ ]:
pid_joint.plot_consistency_bytrialtype(pwdAllH5_2, dfSummary, dropChannels=None, datatype='bn_trial',
                                       trialTypes=['Hit', 'CR'], kind='fisher', fisherThr=0.1, limits=[0, 1])
pid_joint.plot_consistency_bytrialtype(pwdAllH5_2, dfSummary, dropChannels=None, datatype='bn_session',
                                       trialTypes=['Hit', 'CR'], kind='fisher', fisherThr=0.1, limits=[0, 1])

In [ ]:
dfSummary